# Interactive Model Deployment
___________________

## Introduction
___________________

The purpose of this notebook is to demonstrate loading a model from outside of the platform, interactively running that model in a notebook, and visualizing results on top of a basemap through the `Workflows` API. We provide a deploy class, implemented in `utils.py` that manages the models to be deployed as well as fast data retrieval through multi-processing and data caching. 

For the purpose of this demo we will use the NAIP imagery on top of which we deploy a deep learning based computer vision model that detects buildings in the imagery.

You can run the cells in this notebook one at a time by using `Shift-Enter`

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
#import packages
import descarteslabs as dl
import descarteslabs.workflows as wf
import numpy as np
import os
from skimage import measure
from utils import Deploy, rowcol_to_lonlat
from unet import build_model

## Setup
___________________
Next we define some properties which are used for both the visualization of the basemap and data retrieval through the `raster` client. The data product should correspond to the data used to train the model including the bands in the same order as during training.

In [ ]:
# The product the model was trained on. This will also be displayed as base map
product = 'usda:nrcs:naip:rgbn:v1'
resolution = 1.0

# The tilesize defines the image size used for model inference
tilesize = 256
pad = 0

# The bands used to train the model
bands = ['nir', 'red', 'green', 'blue']

In [ ]:
base_img = (
    wf.ImageCollection.from_id(product,
                               start_datetime='2018-01-01',
                               end_datetime='2019-01-01')
    .pick_bands(['red', 'green', 'blue'])
)

In [ ]:
base_img.median(axis='images').visualize('base', scales=[[0, 255], [0, 255], [0, 255]])

## Model loading and deployment
___________________
We will now load the model which is a UNet implemented in Tensorflow2. We have provided the model architecture and a method to build the model in the file `unet.py`. In general the model can be any object that takes a stack of images as input and returns a stack of images of the same size as the input. This is not limited to deep learning models nor to any specific framework. We will give more details on different types of models below.

Running the cell below will download the pre-trained model weights. Note that the provided checkpoint does not correspond to the fully trained model which results in some imperfection of the buildings detection.

In [ ]:
# Download model weights
if not(os.path.exists('model_data_naip.zip')):
    !wget https://storage.googleapis.com/public-published-datasets/model_data_naip.zip
    !unzip model_data_naip.zip

In [ ]:
# Load the model
model = build_model(input_shape=(None, None, 4))

### The deploy class
___________________
Here we will give a little more details on how to use the deploy class. When creating a `Deploy` object a few arguments are required:
```python
class Deploy(map_object, product, resolution, tilesize, bands)
    ''' Deploy class
    map_object: The IPyLeaflet map object
    product: Product from where to retrieve the data
    resolution: The resolution of the raster image
    tilesize: The tilesize which corresponds to the image size for inference
    bands: The bands to be used
    '''
```
Run the cell below to create a `Deploy` object with the settings defined above. This will also add a draw control to the map that allows to define an area of interest to deploy the model. The deployment is triggered once the drawing is complete.

In [ ]:
# Create a deploy object. The model can be passed in here or added later.
deploy = Deploy(wf.map,
                product=product,
                resolution=resolution,
                tilesize=tilesize,
                bands=bands)

### Adding a model
___________________
We can now add the model created above to the `Deploy` object. As mentioned above, the model is in general an object that takes images as input and returns images as outputs. The input images are of shape `(b, h, w, c)` where b = batch_size, h = height, w = width and c = channels. The model should return the target in the shape `(b, h, w, c_t)` where c_t = target channel. If the model is a Tensorflow2 model we can also provide the checkpoint path and optionally the checkpoint to be loaded. The model weights may also be loaded before passing the model to the `Deploy` object if other frameworks are used. Each model added to the `Deploy` object has an associated pre-processing and post-processing function.

`pre_process_fn`: A function that takes images as input and returns the transformed images. A common pre-processing function is mean subtraction and division by the standard deviation to normalize the input images. This is used in the example below. By default the pre-process function returns the original images.

`post_process_fn`: A function that takes the output of the model as input as well as the `dltiles` corresponding to the input images and returns a list of polygons for the detected objects for each layer of the output. If no post-processing function is provided the default post-processing function is used. This applies a threshold of 0.9 to the output and uses the `skimage.measure` method to find contours. The contours are refined through morphological opening and the pixel coordinates are converted to (lat, lon) coordinates using a method provided in `utils.py`.

In [ ]:
deploy.add_model(model,
                 checkpoint_path='model_data_naip/',
                 pre_process_fn=lambda x: (x - 128.0) / 128.0,
                 post_process_fn=None)

### Deploy the model
___________________
We can now **use the draw control on the map** to define an area of interest to deploy the model over. Note that if we had added more models, each model would be deployed over that area and the output shown on top of the basemap.

The `Deploy` class makes use of multiprocessing to download the imagery in the area of interest. It downloads a maximum of batch_size at once where batch_size can be defined throught the `max_batch_size` argument of the `Deploy` object. It then runs batch inference of all models before retrieving the next batch. The downloaded images are kept in an internal cache wich makes re-deployment over the same area even faster.

In [ ]:
# Display the map
wf.map.center = 37.7770, -122.4409 # San Francisco
wf.map.zoom = 14
wf.map

The `Deploy` class also keeps track of some computation times. We can print these times by running the cell below. As this is the first time we deployed the model, most of the time will be spent in the `raster` calls and if you run this notebook on a CPU in model inference.

In [ ]:
deploy.print_compute_times()

### Adding additional models
___________________
The main purpose of the `Deploy` class is to quickly deploy a model and inspect its output. This is particularely useful if different models are tested as it allows to compare the outputs. We may also want to add different models that were trained to detect different objects and show their output in the same map.

An example for a non deep learning model is a simple function such as
```python
def my_function(images):
    # do something with images
    return outputs
```
We leave it up to the user to implement their own models and deploy them with this approach. When adding additional models, they can be given specific names through the `model_name` argument which allows managing the different models such as removing them from the 